In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import os
import glob

current_dir= Path.cwd()
reference_data_dir= current_dir.parent/'reference_data'
print(reference_data_dir)

/Users/keenansmacbookairm3/Documents/GitHub/SnowDay_NYC/python-statement/reference_data


In [2]:
glob.glob(f'{reference_data_dir}/*119*.csv')[0]

IndexError: list index out of range

In [20]:
house_119_df = pd.read_csv((f'{reference_data_dir}/119_congress_full_list.csv'))
house_119_df.head()

,state,district,name,party,house_page
0,Alabama,1st,"Moore, Barry",R,https://barrymoore.house.gov
1,Alabama,2nd,"Figures, Shomari",D,https://figures.house.gov/
2,Alabama,3rd,"Rogers, Mike",R,https://mikerogers.house.gov/
3,Alabama,4th,"Aderholt, Robert",R,https://aderholt.house.gov/
4,Alabama,5th,"Strong, Dale",R,https://strong.house.gov


In [21]:
import tldextract


def url_cleanup(url: str) -> str:
    parsed = tldextract.extract(url)

    # Remove 'www' only if it's the left-most subdomain
    subdomain = parsed.subdomain
    if subdomain.startswith("www."):
        subdomain = subdomain[4:]
  

    parts = [subdomain, parsed.domain, parsed.suffix]
    return ".".join(part for part in parts if part)


In [22]:
house_119_df['domain'] = house_119_df.house_page.apply(url_cleanup)
house_119_df.head()

,state,district,name,party,house_page,domain
0,Alabama,1st,"Moore, Barry",R,https://barrymoore.house.gov,barrymoore.house.gov
1,Alabama,2nd,"Figures, Shomari",D,https://figures.house.gov/,figures.house.gov
2,Alabama,3rd,"Rogers, Mike",R,https://mikerogers.house.gov/,mikerogers.house.gov
3,Alabama,4th,"Aderholt, Robert",R,https://aderholt.house.gov/,aderholt.house.gov
4,Alabama,5th,"Strong, Dale",R,https://strong.house.gov,strong.house.gov


In [23]:
from python_statement import Scraper

In [24]:
for k, v in Scraper().SCRAPER_CONFIG.items():
    print(f'{k} ---> {v}')
    print('-------')

moran ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.moran.senate.gov/public/index.cfm/news-releases'}
-------
boozman ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.boozman.senate.gov/public/index.cfm/press-releases'}
-------
thune ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.thune.senate.gov/public/index.cfm/press-releases'}
-------
barrasso ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.barrasso.senate.gov/public/index.cfm/news-releases'}
-------
graham ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.lgraham.senate.gov/public/index.cfm/press-releases'}
-------
klobuchar ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.klobuchar.senate.gov/public/index.cfm/news-releases'}
-------
mcconnell ---> {'method': 'table_recordlist_date', 'url_base': 'https://www.mcconnell.senate.gov/public/index.cfm/pressreleases'}
-------
timscott ---> {'method': 'jet_listing_elementor', 'url_bas

In [25]:
scraper_config_summary = pd.DataFrame(Scraper.SCRAPER_CONFIG).T
scraper_config_summary

,method,url_base
moran,table_recordlist_date,https://www.moran.senate.gov/public/index.cfm/news-releases
boozman,table_recordlist_date,https://www.boozman.senate.gov/public/index.cfm/press-releases
thune,table_recordlist_date,https://www.thune.senate.gov/public/index.cfm/press-releases
barrasso,table_recordlist_date,https://www.barrasso.senate.gov/public/index.cfm/news-releases
graham,table_recordlist_date,https://www.lgraham.senate.gov/public/index.cfm/press-releases
...,...,...
vindman,media_body,https://vindman.house.gov/media/press-releases
subramanyam,media_body,https://subramanyam.house.gov/media/press-releases
baumgartner,media_body,https://baumgartner.house.gov/media/press-releases
randall,media_body,https://randall.house.gov/media/press-releases


In [26]:
extraction = url_cleanup('https://www.moran.senate.gov/public/index.cfm/news-releases')
extraction

'moran.senate.gov'

In [27]:
scraper_config_summary['domain'] = scraper_config_summary.url_base.apply(url_cleanup)
scraper_config_summary.head()

,method,url_base,domain
moran,table_recordlist_date,https://www.moran.senate.gov/public/index.cfm/news-releases,moran.senate.gov
boozman,table_recordlist_date,https://www.boozman.senate.gov/public/index.cfm/press-releases,boozman.senate.gov
thune,table_recordlist_date,https://www.thune.senate.gov/public/index.cfm/press-releases,thune.senate.gov
barrasso,table_recordlist_date,https://www.barrasso.senate.gov/public/index.cfm/news-releases,barrasso.senate.gov
graham,table_recordlist_date,https://www.lgraham.senate.gov/public/index.cfm/press-releases,lgraham.senate.gov


In [28]:
house_scraper_config_summary = scraper_config_summary[scraper_config_summary.domain.str.contains('house.gov')]
house_scraper_config_summary.shape

(271, 3)

In [29]:
house_119_df['domain_available'] = 'no'


In [30]:
overlap_mask = house_119_df["domain"].isin(house_scraper_config_summary["domain"])
house_119_df.loc[overlap_mask, 'domain_available'] = 'yes'

house_119_df.domain_available.value_counts()

domain_available
yes    265
no     176
Name: count, dtype: int64

In [31]:
house_119_df[house_119_df.domain_available == 'no']

,state,district,name,party,house_page,domain,domain_available
2,Alabama,3rd,"Rogers, Mike",R,https://mikerogers.house.gov/,mikerogers.house.gov,no
6,Alabama,7th,"Sewell, Terri",D,https://sewell.house.gov/,sewell.house.gov,no
9,Arizona,1st,"Schweikert, David",R,https://schweikert.house.gov/,schweikert.house.gov,no
10,Arizona,2nd,"Crane, Elijah",R,https://crane.house.gov,crane.house.gov,no
12,Arizona,4th,"Stanton, Greg",D,https://stanton.house.gov/,stanton.house.gov,no
...,...,...,...,...,...,...,...
422,Washington,3rd,"Perez, Marie",D,https://gluesenkampperez.house.gov,gluesenkampperez.house.gov,no
426,Washington,7th,"Jayapal, Pramila",D,https://jayapal.house.gov,jayapal.house.gov,no
429,Washington,10th,"Strickland, Marilyn",D,https://strickland.house.gov,strickland.house.gov,no
435,Wisconsin,4th,"Moore, Gwen",D,https://gwenmoore.house.gov,gwenmoore.house.gov,no


In [ ]:
house_119_df